In [1]:
import pandas as pd
import pysam

In [2]:
col_list = ["Transcript_ID","Gene_ID","Gene_name","5UTR_length","CDS_length","3UTR_length"]
P_gtf = pd.read_csv('/Data_2/Daehwa/Data_Library/GTF_parsed/v0.7.1/gencode.vM27.annotation.gtf/Processed_gtf.tsv', sep='\t', usecols=col_list)
repre = pd.read_csv('/Data_2/Jun/Adipocytes/references/representative-isoforms.txt', sep='\t', names=['Gene_ID','Transcript_ID','a','b','c']).drop(columns=['a','b','c'])

RPF_abundant = pd.read_csv('/Data_2/Daehwa/Adipocyte/Other_papers/Martinez_etal.2023.Cell_Metabolism/Analysis/Gene_lists/Abundant_genes/v20230706/Martinez_RPF_top100_abundant_genelist.txt', names=['gene_id'])
# RNA_abundant = pd.read_csv('/Data_2/Daehwa/Adipocyte/Analysis/Gene_lists/Abundant_genes/v20230529/adi_RNA_top100_abundant_genelist.txt', names=['gene_id'])

In [3]:
DBs = {}

DB = pd.merge(repre, P_gtf, on=['Gene_ID','Transcript_ID'])
DB = DB.rename(columns = {'Gene_ID':'gene_id',
                          'Gene_name':'gene_name',
                          'Transcript_ID':'transcript_id'})
DB = pd.merge(RPF_abundant, DB, on='gene_id')
DB = DB[['transcript_id',"5UTR_length","CDS_length","3UTR_length","gene_id","gene_name"]]
DB = DB[ DB['CDS_length'] > 300 ]
DB = DB.set_index('transcript_id').T.to_dict('list')

DBs['RPF'] = DB

# DB = pd.merge(repre, P_gtf, on=['Gene_ID','Transcript_ID'])
# DB = DB.rename(columns = {'Gene_ID':'gene_id',
#                           'Gene_name':'gene_name',
#                           'Transcript_ID':'transcript_id'})
# DB = pd.merge(RNA_abundant, DB, on='gene_id')
# DB = DB[['transcript_id',"5UTR_length","CDS_length","3UTR_length","gene_id","gene_name"]]
# DB = DB[ DB['CDS_length'] > 300 ]
# DB = DB.set_index('transcript_id').T.to_dict('list')

# DBs['RNA'] = DB

display(dict(list(DB.items())[:20]))

{'ENSMUST00000001547.8': [145, 4362, 1423, 'ENSMUSG00000001506.11', 'Col1a1'],
 'ENSMUST00000031668.10': [347, 4119, 882, 'ENSMUSG00000029661.17', 'Col1a2'],
 'ENSMUST00000087883.13': [210, 4395, 959, 'ENSMUSG00000026043.19', 'Col3a1'],
 'ENSMUST00000029041.6': [65, 399, 432, 'ENSMUSG00000062515.4', 'Fabp4'],
 'ENSMUST00000041331.4': [300, 1068, 3464, 'ENSMUSG00000037071.4', 'Scd1'],
 'ENSMUST00000073012.13': [18, 2436, 735, 'ENSMUSG00000027750.17', 'Postn'],
 'ENSMUST00000055655.9': [527, 7515, 2008, 'ENSMUSG00000025153.10', 'Fasn'],
 'ENSMUST00000214685.2': [196, 909, 164, 'ENSMUSG00000018593.14', 'Sparc'],
 'ENSMUST00000015712.15': [262, 1425, 2428, 'ENSMUSG00000015568.17', 'Lpl'],
 'ENSMUST00000039559.9': [228, 3516, 2104, 'ENSMUSG00000040152.9', 'Thbs1'],
 'ENSMUST00000041544.8': [459, 3666, 661, 'ENSMUSG00000064080.13', 'Fbln2'],
 'ENSMUST00000055226.13': [193, 7434, 688, 'ENSMUSG00000026193.16', 'Fn1'],
 'ENSMUST00000097653.11': [235, 8034, 732, 'ENSMUSG00000048126.17', 'Col6a3'

In [4]:
prefix = 'Martinez_'
#data file
folder = '/Data_2/Daehwa/Adipocyte/Other_papers/Martinez_etal.2023.Cell_Metabolism/Alignment/tr-aln'
dataset = {'RPF':'RPF'}
Conds = ['BG','BR','WH']
reps = ['a','b']
filetype = ".rep.bam"

In [6]:
out_w = pd.ExcelWriter(prefix+'abundant_metagene.xlsx')
for LIB in ['RPF']:
    for Cond in Conds:
        for rep in reps:
            SP = Cond+rep
            result_start = {}
            result_stop = {}
            result_start['transcript_id'] = [str(i) for i in range(-100,201)]
            result_stop['transcript_id'] = [str(i) for i in range(-200,101)]
            
            for T_id in DBs[LIB]:
                result_start[T_id] = [0]*len(range(-100,201))
                result_stop[T_id]  = [0]*len(range(-200,101))
            
            f = pysam.AlignmentFile(f'{folder}/{dataset[LIB]}/{prefix}{SP}{filetype}','rb')
            for line in f:
                line = line.tostring(f)
                Aligned = line.split("\t")
                
                T_id = Aligned[2]
                
                if DBs[LIB].get(T_id)==None: continue
                P = DBs[LIB][T_id]

                start_pos = int( int(Aligned[3])-(P[0]+1) )
                stop_pos = int( int(Aligned[3])-(P[0]+P[1]-2) )
                if -100 <= start_pos <= 200 :
                    result_start[T_id][start_pos+100] += 1
                if -200 <= stop_pos <= 100 :
                    result_stop[T_id][stop_pos+200] += 1
            
            print (f'{LIB} {SP} done')

            result_start = pd.DataFrame(result_start).T
            result_start.to_excel(out_w, sheet_name=f'{LIB}_{SP}_start', header=False)
            result_stop = pd.DataFrame(result_stop).T
            result_stop.to_excel(out_w, sheet_name=f'{LIB}_{SP}_stop', header=False)
out_w.close()

RPF BGa done
RPF BGb done
RPF BRa done
RPF BRb done
RPF WHa done
RPF WHb done
